<a href="https://colab.research.google.com/github/Samruddhirajesh/Demand-Prediction-for-public-transport/blob/main/Transport_Demand_in_nairobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/train_revised.csv')

In [ ]:
dataset.head()

,ride_id,seat_number,payment_method,payment_receipt,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity
0,1442,15A,Mpesa,UZUEHCBUSO,17-10-17,7:15,Migori,Nairobi,Bus,49
1,5437,14A,Mpesa,TIHLBUSGTE,19-11-17,7:12,Migori,Nairobi,Bus,49
2,5710,8B,Mpesa,EQX8Q5G19O,26-11-17,7:05,Keroka,Nairobi,Bus,49
3,5777,19A,Mpesa,SGP18CL0ME,27-11-17,7:10,Homa Bay,Nairobi,Bus,49
4,5778,11A,Mpesa,BM97HFRGL9,27-11-17,7:12,Migori,Nairobi,Bus,49


In [ ]:
dataset.tail()

,ride_id,seat_number,payment_method,payment_receipt,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity
51640,13826,9B,Mpesa,8V2XDDZR6V,20-04-18,8:00,Awendo,Nairobi,Bus,49
51641,13809,18A,Mpesa,4PEBSVJSNK,20-04-18,8:00,Migori,Nairobi,Bus,49
51642,13809,17A,Mpesa,LVN64LZDNN,20-04-18,8:00,Migori,Nairobi,Bus,49
51643,13796,16B,Mpesa,REYBSKTYWN,20-04-18,7:08,Awendo,Nairobi,Bus,49
51644,14304,7,Mpesa,AQN7FBUSGP,14-11-17,5:10,Kisii,Nairobi,Bus,49


In [ ]:
dataset.shape

(51645, 10)

There are 51645 observations in the dataset. And has 10 columns.

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51645 entries, 0 to 51644
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ride_id          51645 non-null  int64 
 1   seat_number      51645 non-null  object
 2   payment_method   51645 non-null  object
 3   payment_receipt  51645 non-null  object
 4   travel_date      51645 non-null  object
 5   travel_time      51645 non-null  object
 6   travel_from      51645 non-null  object
 7   travel_to        51645 non-null  object
 8   car_type         51645 non-null  object
 9   max_capacity     51645 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 3.9+ MB


In [ ]:
dataset.describe(include='object')

,seat_number,payment_method,payment_receipt,travel_date,travel_time,travel_from,travel_to,car_type
count,51645,51645,51645,51645,51645,51645,51645,51645
unique,61,2,51645,149,78,17,1,2
top,1,Mpesa,UZUEHCBUSO,10-12-17,7:09,Kisii,Nairobi,Bus
freq,2065,51532,1,856,3926,22607,51645,31985


Summary:-
There are total 61 unique seats in this dataset

travelers have used 2 types of payment method and most of the people have used Mpesa to pay for their ticket.

The record of 149 days out of 2 year is present in this dataset.

There are 2 different types of car and most of them are bus.

In [ ]:
dataset.isnull().sum()

ride_id            0
seat_number        0
payment_method     0
payment_receipt    0
travel_date        0
travel_time        0
travel_from        0
travel_to          0
car_type           0
max_capacity       0
dtype: int64

In [ ]:
len(dataset['ride_id'].unique())

6249

we will find the count of each ride_id and that will be the number_of_ticket as our target variable.

In [ ]:
label=dataset.groupby(["ride_id"]).seat_number.count().rename("number_of_ticket").reset_index()
label.head()

,ride_id,number_of_ticket
0,1442,1
1,5437,1
2,5710,1
3,5777,5
4,5778,31


Now in our target variable delete the duplicate values from ride_id

In [ ]:
dataset=dataset.drop_duplicates("ride_id")

In [ ]:
dataset.shape

(6249, 10)

Now let us merge two datasets on ride_id key to make the final dataset where we have dependent as well as independent variables too.

In [ ]:
dataset = dataset.merge(label, how="left",on="ride_id")
dataset.head()

,ride_id,seat_number,payment_method,payment_receipt,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity,number_of_ticket
0,1442,15A,Mpesa,UZUEHCBUSO,17-10-17,7:15,Migori,Nairobi,Bus,49,1
1,5437,14A,Mpesa,TIHLBUSGTE,19-11-17,7:12,Migori,Nairobi,Bus,49,1
2,5710,8B,Mpesa,EQX8Q5G19O,26-11-17,7:05,Keroka,Nairobi,Bus,49,1
3,5777,19A,Mpesa,SGP18CL0ME,27-11-17,7:10,Homa Bay,Nairobi,Bus,49,5
4,5778,11A,Mpesa,BM97HFRGL9,27-11-17,7:12,Migori,Nairobi,Bus,49,31


Now we have found our final dataset so let us remove some features that doesn't seem to be important for predicting the number of tickets in our model

In [ ]:
dataset = dataset.drop(['seat_number','payment_method','payment_receipt'], axis=1)

In [ ]:
#concatinate travel date and travel time column and make a new feature called date.
dataset["date"]=dataset["travel_date"]+ " "+dataset["travel_time"]

In [ ]:
def  time_features(df):

  '''This function takes dataframe as an argument and extracts the
  different features from the date variable of the dataset and finaly returns the updated
  dataset'''

  df["date"]=pd.to_datetime(df["date"])
  df["day_of_week"]=df["date"].dt.dayofweek
  df["day_of_year"]=df["date"].dt.dayofyear
  df["day_of_month"]=df["date"].dt.day
  df["year_woy"]=df["date"].dt.year.astype(str)+df["date"].dt.weekofyear.astype(str)
  df["hour"]=df["date"].dt.hour
  df["minute"]=df["date"].dt.minute

  df["is_weekend"]=df["day_of_week"].apply( lambda x : 1 if x  in [5,6] else 0 )  # A lambda function that find if a day is weekend
  df["year"]=df["date"].dt.year
  df["quarter"]=df["date"].dt.quarter
  df["month"]=df["date"].dt.month
  return df

In [ ]:
dataset_new = time_features(dataset)

<ipython-input-17-964f76ab4c7a>:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df["year_woy"]=df["date"].dt.year.astype(str)+df["date"].dt.weekofyear.astype(str)


In [ ]:
dataset_new.head()

,ride_id,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity,number_of_ticket,date,day_of_week,day_of_year,day_of_month,year_woy,hour,minute,is_weekend,year,quarter,month
0,1442,17-10-17,7:15,Migori,Nairobi,Bus,49,1,2017-10-17 07:15:00,1,290,17,201742,7,15,0,2017,4,10
1,5437,19-11-17,7:12,Migori,Nairobi,Bus,49,1,2017-11-19 07:12:00,6,323,19,201746,7,12,1,2017,4,11
2,5710,26-11-17,7:05,Keroka,Nairobi,Bus,49,1,2017-11-26 07:05:00,6,330,26,201747,7,5,1,2017,4,11
3,5777,27-11-17,7:10,Homa Bay,Nairobi,Bus,49,5,2017-11-27 07:10:00,0,331,27,201748,7,10,0,2017,4,11
4,5778,27-11-17,7:12,Migori,Nairobi,Bus,49,31,2017-11-27 07:12:00,0,331,27,201748,7,12,0,2017,4,11


In [ ]:
dataset_new["travel_time"] = dataset_new["travel_time"].str.split(':').apply(lambda x: int(x[0]) + int(x[1])/60)

In [ ]:
dataset_new['period'] = np.nan

dataset_new.loc[dataset_new.travel_time < 7, 'period'] = 'em'
dataset_new.loc[(dataset_new.travel_time >= 7) & (dataset_new.travel_time < 11), 'period'] = 'am'
dataset_new.loc[(dataset_new.travel_time >= 11) & (dataset_new.travel_time < 15), 'period'] = 'mid'
dataset_new.loc[(dataset_new.travel_time >= 15) & (dataset_new.travel_time < 19), 'period'] = 'eve'
dataset_new.loc[(dataset_new.travel_time >= 19) & (dataset_new.travel_time <= 24), 'period'] = 'pm'


pcount = dict(dataset_new['period'].value_counts())
pcount


dataset_new['hourly_travelers'] = np.log1p(dataset_new['period'].map(pcount))



dcount = dict(dataset_new["day_of_year"].value_counts())
dataset_new["daily_travelers"] = np.log1p(dataset_new["day_of_year"].map(dcount))
dataset_new.head()

,ride_id,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity,number_of_ticket,date,day_of_week,...,year_woy,hour,minute,is_weekend,year,quarter,month,period,hourly_travelers,daily_travelers
0,1442,17-10-17,7.250000,Migori,Nairobi,Bus,49,1,2017-10-17 07:15:00,1,...,201742,7,15,0,2017,4,10,am,8.494743,0.693147
1,5437,19-11-17,7.200000,Migori,Nairobi,Bus,49,1,2017-11-19 07:12:00,6,...,201746,7,12,1,2017,4,11,am,8.494743,0.693147
2,5710,26-11-17,7.083333,Keroka,Nairobi,Bus,49,1,2017-11-26 07:05:00,6,...,201747,7,5,1,2017,4,11,am,8.494743,0.693147
3,5777,27-11-17,7.166667,Homa Bay,Nairobi,Bus,49,5,2017-11-27 07:10:00,0,...,201748,7,10,0,2017,4,11,am,8.494743,3.871201
4,5778,27-11-17,7.200000,Migori,Nairobi,Bus,49,31,2017-11-27 07:12:00,0,...,201748,7,12,0,2017,4,11,am,8.494743,3.871201


In [ ]:
dataset_new.columns

Index(['ride_id', 'travel_date', 'travel_time', 'travel_from', 'travel_to',
       'car_type', 'max_capacity', 'number_of_ticket', 'date', 'day_of_week',
       'day_of_year', 'day_of_month', 'year_woy', 'hour', 'minute',
       'is_weekend', 'year', 'quarter', 'month', 'period', 'hourly_travelers',
       'daily_travelers'],
      dtype='object')

In [ ]:
new_cols = ['day_of_year','daily_travelers','minute','day_of_month','hourly_travelers','is_weekend','day_of_week']

In [ ]:
transport_dataset=dataset_new.copy()

In [ ]:
transport_dataset.head()

,ride_id,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity,number_of_ticket,date,day_of_week,...,year_woy,hour,minute,is_weekend,year,quarter,month,period,hourly_travelers,daily_travelers
0,1442,17-10-17,7.250000,Migori,Nairobi,Bus,49,1,2017-10-17 07:15:00,1,...,201742,7,15,0,2017,4,10,am,8.494743,0.693147
1,5437,19-11-17,7.200000,Migori,Nairobi,Bus,49,1,2017-11-19 07:12:00,6,...,201746,7,12,1,2017,4,11,am,8.494743,0.693147
2,5710,26-11-17,7.083333,Keroka,Nairobi,Bus,49,1,2017-11-26 07:05:00,6,...,201747,7,5,1,2017,4,11,am,8.494743,0.693147
3,5777,27-11-17,7.166667,Homa Bay,Nairobi,Bus,49,5,2017-11-27 07:10:00,0,...,201748,7,10,0,2017,4,11,am,8.494743,3.871201
4,5778,27-11-17,7.200000,Migori,Nairobi,Bus,49,31,2017-11-27 07:12:00,0,...,201748,7,12,0,2017,4,11,am,8.494743,3.871201


In [ ]:
transport_dataset.shape

(6249, 22)

In [ ]:
transport_dataset.month.value_counts()

12    1050
2      985
1      896
3      853
4      600
11     414
9      262
7      254
8      251
10     247
6      225
5      212
Name: month, dtype: int64

In [ ]:
dict_cov = {12:1,2:1,1:1,2:1,4:2,11:2,9:3,7:3,8:3,10:3,6:3,5:3}
transport_dataset['month'].replace(dict_cov,inplace=True)

transport_dataset.month.value_counts()

1    2931
3    2304
2    1014
Name: month, dtype: int64

In [ ]:
transport_dataset.day_of_month.unique()

array([17, 19, 26, 27, 20, 18, 16, 15, 14, 13,  4, 28, 31, 30, 29, 25, 24,
       23, 22, 21, 12,  3,  2,  1])

In [ ]:
conv_dict = {17:2, 19:2, 26:3, 27:3, 20:2, 18:2, 16:2, 15:2, 14:2, 13:2,  4:1, 28:2, 31:3, 30:3, 29:3, 25:3, 24:3,
       23:3, 22:3, 21:3, 12:1,  3:1,  2:1,  1:2}
transport_dataset['day_of_month'].replace(conv_dict,inplace=True)

In [ ]:
transport_dataset['day_of_month'].value_counts()

1    2496
2    2305
3    1448
Name: day_of_month, dtype: int64

In [ ]:
transport_dataset.head()

,ride_id,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity,number_of_ticket,date,day_of_week,...,year_woy,hour,minute,is_weekend,year,quarter,month,period,hourly_travelers,daily_travelers
0,1442,17-10-17,7.250000,Migori,Nairobi,Bus,49,1,2017-10-17 07:15:00,1,...,201742,7,15,0,2017,4,3,am,8.494743,0.693147
1,5437,19-11-17,7.200000,Migori,Nairobi,Bus,49,1,2017-11-19 07:12:00,6,...,201746,7,12,1,2017,4,2,am,8.494743,0.693147
2,5710,26-11-17,7.083333,Keroka,Nairobi,Bus,49,1,2017-11-26 07:05:00,6,...,201747,7,5,1,2017,4,2,am,8.494743,0.693147
3,5777,27-11-17,7.166667,Homa Bay,Nairobi,Bus,49,5,2017-11-27 07:10:00,0,...,201748,7,10,0,2017,4,2,am,8.494743,3.871201
4,5778,27-11-17,7.200000,Migori,Nairobi,Bus,49,31,2017-11-27 07:12:00,0,...,201748,7,12,0,2017,4,2,am,8.494743,3.871201


In [ ]:
# function to find the characterstics of the variable in the dataframe.
def data_characterization(df):
  '''This function takes dataframe as an argument and returns
    the characterstics of the features of the dataframe'''
  characterized_df = pd.DataFrame(columns=['Columns name',	'Type', 'unique values Count', 'Count Nan values','Ratio Nan values','Unique values',	'Max',	'Min','Mean','std'])
  for col in df.columns:
    try:
      characterized_df = characterized_df.append({'Columns name':col,	'Type':df[col].dtype, 'unique values Count':len(df[col].unique()),
                                                  'Count Nan values':df[col].isnull().sum(),'Ratio Nan values':df[col].isnull().sum()/df[col].shape[0],
                                                  'Unique values':df[col].unique(),	'Max':df[col].max(),	'Min':df[col].min(),'Mean':df[col].mean(),'std':df[col].std()},
                                                ignore_index=True)
    except:
      characterized_df = characterized_df.append({'Columns name':col,	'Type':df[col].dtype, 'unique values Count':len(df[col].unique()),
                                                  'Count Nan values':df[col].isnull().sum(),'Ratio Nan values':df[col].isnull().sum()/df[col].shape[0],
                                                  'Unique values':df[col].unique(),	'Max':np.nan,	'Min':np.nan,'Mean':np.nan,'std':np.nan},
                                                ignore_index=True)
  return characterized_df

In [ ]:
data_characterization(transport_dataset)


<ipython-input-33-f935e821a23d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  characterized_df = characterized_df.append({'Columns name':col,	'Type':df[col].dtype, 'unique values Count':len(df[col].unique()),
<ipython-input-33-f935e821a23d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  characterized_df = characterized_df.append({'Columns name':col,	'Type':df[col].dtype, 'unique values Count':len(df[col].unique()),
<ipython-input-33-f935e821a23d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  characterized_df = characterized_df.append({'Columns name':col,	'Type':df[col].dtype, 'unique values Count':len(df[col].unique()),
<ipython-input-33-f935e821a23d>:13: FutureWarning: The frame.append method is deprecated and wil

,Columns name,Type,unique values Count,Count Nan values,Ratio Nan values,Unique values,Max,Min,Mean,std
0,ride_id,int64,6249,0,0.0,"[1442, 5437, 5710, 5777, 5778, 5781, 5784, 578...",20117,1442,9963.644583,2296.304872
1,travel_date,object,149,0,0.0,"[17-10-17, 19-11-17, 26-11-17, 27-11-17, 20-04...",NaN,NaN,NaN,NaN
2,travel_time,float64,78,0,0.0,"[7.25, 7.2, 7.083333333333333, 7.1666666666666...",23.166667,5.0,8.239828,2.951814
3,travel_from,object,17,0,0.0,"[Migori, Keroka, Homa Bay, Kisii, Keumbu, Rong...",NaN,NaN,NaN,NaN
4,travel_to,object,1,0,0.0,[Nairobi],NaN,NaN,NaN,NaN
5,car_type,object,2,0,0.0,"[Bus, shuttle]",NaN,NaN,NaN,NaN
6,max_capacity,int64,2,0,0.0,"[49, 11]",49,11,30.392223,18.997471
7,number_of_ticket,int64,50,0,0.0,"[1, 5, 31, 26, 2, 3, 11, 4, 9, 6, 8, 10, 24, 7...",50,1,8.264522,8.632968
8,date,datetime64[ns],4579,0,0.0,"[2017-10-17T07:15:00.000000000, 2017-11-19T07:...",2018-12-04 23:10:00,2017-01-12 05:00:00,2018-03-03 16:51:43.226116352,128 days 10:31:39.159274134
9,day_of_week,int64,7,0,0.0,"[1, 6, 0, 4, 3, 2, 5]",6,0,2.913906,1.958013


In [ ]:
# Let us find the difference between next and previous buses
# And count the travel per minut trip
def find_difference_bw_bus(data):

  data.sort_values(["travel_from","date"],inplace=True,ascending=True)
  data["diff_btw_0_1_next_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(-1)).dt.total_seconds()/3600
  data["diff_btw_0_1_previous_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(1)).dt.total_seconds()/3600
  data["diff_btw_0_2_next_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(-2)).dt.total_seconds()/3600
  data["diff_btw_0_2_previous_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(2)).dt.total_seconds()/3600
  data["diff_btw_0_3_next_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(-3)).dt.total_seconds()/3600
  data["diff_btw_0_3_previous_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(3)).dt.total_seconds()/3600
  data["diff_btw_next_previous_bus"]=(data.groupby(["travel_from"]).date.shift(-1)-data.groupby(["travel_from"]).date.shift(1)).dt.total_seconds()/3600
  fea=['diff_btw_0_1_next_bus', 'diff_btw_0_1_previous_bus', 'diff_btw_0_2_next_bus', 'diff_btw_0_2_previous_bus',
      'diff_btw_0_3_next_bus', 'diff_btw_0_3_previous_bus',
      'diff_btw_next_previous_bus']
  data[fea]=data.groupby(["travel_from"])[fea].fillna(method="ffill")
  data[fea]=data.groupby(["travel_from"])[fea].fillna(method="backfill")

  for i in [1,3,5,7,10,15] :
      data["count_trip_per_{}min_travel_from".format(str(i))]=data.groupby([pd.Grouper(key="date",freq='{}min'.format(i)),"travel_from"]).ride_id.transform("count")
      data["count_trip_per_{}min".format(str(i))]=data.groupby([pd.Grouper(key="date",freq='{}min'.format(i))]).ride_id.transform("count")

  return data

In [ ]:
data = find_difference_bw_bus(transport_dataset)

In [ ]:
data_characterization(data)

<ipython-input-33-f935e821a23d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  characterized_df = characterized_df.append({'Columns name':col,	'Type':df[col].dtype, 'unique values Count':len(df[col].unique()),
<ipython-input-33-f935e821a23d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  characterized_df = characterized_df.append({'Columns name':col,	'Type':df[col].dtype, 'unique values Count':len(df[col].unique()),
<ipython-input-33-f935e821a23d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  characterized_df = characterized_df.append({'Columns name':col,	'Type':df[col].dtype, 'unique values Count':len(df[col].unique()),
<ipython-input-33-f935e821a23d>:13: FutureWarning: The frame.append method is deprecated and wil

,Columns name,Type,unique values Count,Count Nan values,Ratio Nan values,Unique values,Max,Min,Mean,std
0,ride_id,int64,6249,0,0.00000,"[6654, 12926, 12929, 12935, 12924, 8627, 8628,...",20117,1442,9963.644583,2296.304872
1,travel_date,object,149,0,0.00000,"[05-12-17, 01-04-18, 13-01-18, 14-01-18, 15-01...",NaN,NaN,NaN,NaN
2,travel_time,float64,78,0,0.00000,"[7.133333333333334, 7.2, 7.216666666666667, 8....",23.166667,5.0,8.239828,2.951814
3,travel_from,object,17,0,0.00000,"[Awendo, Homa Bay, Kehancha, Kendu Bay, Keroka...",NaN,NaN,NaN,NaN
4,travel_to,object,1,0,0.00000,[Nairobi],NaN,NaN,NaN,NaN
5,car_type,object,2,0,0.00000,"[Bus, shuttle]",NaN,NaN,NaN,NaN
6,max_capacity,int64,2,0,0.00000,"[49, 11]",49,11,30.392223,18.997471
7,number_of_ticket,int64,50,0,0.00000,"[2, 20, 9, 15, 6, 1, 4, 5, 8, 12, 3, 17, 18, 1...",50,1,8.264522,8.632968
8,date,datetime64[ns],4579,0,0.00000,"[2017-05-12T07:08:00.000000000, 2018-01-04T07:...",2018-12-04 23:10:00,2017-01-12 05:00:00,2018-03-03 16:51:43.226116096,128 days 10:31:39.159274134
9,day_of_week,int64,7,0,0.00000,"[4, 3, 5, 6, 0, 1, 2]",6,0,2.913906,1.958013


In [ ]:
data.dropna(inplace=True)

In [ ]:
data.head()

,ride_id,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity,number_of_ticket,date,day_of_week,...,count_trip_per_3min_travel_from,count_trip_per_3min,count_trip_per_5min_travel_from,count_trip_per_5min,count_trip_per_7min_travel_from,count_trip_per_7min,count_trip_per_10min_travel_from,count_trip_per_10min,count_trip_per_15min_travel_from,count_trip_per_15min
502,6654,05-12-17,7.133333,Awendo,Nairobi,Bus,49,2,2017-05-12 07:08:00,4,...,1,7,1,11,1,12,1,16,1,21
5530,12926,01-04-18,7.133333,Awendo,Nairobi,Bus,49,20,2018-01-04 07:08:00,3,...,1,6,1,8,3,11,1,12,3,20
5533,12929,01-04-18,7.200000,Awendo,Nairobi,Bus,49,2,2018-01-04 07:12:00,3,...,2,7,2,8,3,11,2,8,3,20
5539,12935,01-04-18,7.216667,Awendo,Nairobi,Bus,49,9,2018-01-04 07:13:00,3,...,2,7,2,8,3,11,2,8,3,20
5528,12924,01-04-18,8.000000,Awendo,Nairobi,Bus,49,15,2018-01-04 08:00:00,3,...,1,2,1,2,1,2,1,2,1,2


In [ ]:
from datetime import datetime
from datetime import timedelta

In [ ]:
# The list of hollydays in Nirobi

# The  main goal of this chunk of the code is to find that there is holiday at perticular date or not

holidays_date=[datetime(2018,1,1),
              datetime(2018,3,30),
              datetime(2018,4,2),
              datetime(2018,6,1),
              datetime(2018,6,15),
              datetime(2017,10,20),
              datetime(2017,11,7),
              datetime(2017,12,12),
              datetime(2017,12,25),
              datetime(2017,12,26)]

holidays_date_mins_one_day=[ (day- timedelta(days=1)).date() for day in holidays_date]
holidays_date_mins_two_day=[ (day- timedelta(days=2)).date() for day in holidays_date]

holidays_date_plus_one_day=[ (day+ timedelta(days=1)).date() for day in holidays_date]
holidays_date=[ day.date() for day in holidays_date]

data["is_holidays"]=0
data["is_after_toomorw_holidays"]=0


data.loc[data.date.dt.date.isin(holidays_date),"is_holidays"]=1
data.loc[data.date.dt.date.isin(holidays_date_mins_two_day),"is_after_toomorw_holidays"]=1

In [ ]:
data.head()

,ride_id,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity,number_of_ticket,date,day_of_week,...,count_trip_per_5min_travel_from,count_trip_per_5min,count_trip_per_7min_travel_from,count_trip_per_7min,count_trip_per_10min_travel_from,count_trip_per_10min,count_trip_per_15min_travel_from,count_trip_per_15min,is_holidays,is_after_toomorw_holidays
502,6654,05-12-17,7.133333,Awendo,Nairobi,Bus,49,2,2017-05-12 07:08:00,4,...,1,11,1,12,1,16,1,21,0,0
5530,12926,01-04-18,7.133333,Awendo,Nairobi,Bus,49,20,2018-01-04 07:08:00,3,...,1,8,3,11,1,12,3,20,0,0
5533,12929,01-04-18,7.200000,Awendo,Nairobi,Bus,49,2,2018-01-04 07:12:00,3,...,2,8,3,11,2,8,3,20,0,0
5539,12935,01-04-18,7.216667,Awendo,Nairobi,Bus,49,9,2018-01-04 07:13:00,3,...,2,8,3,11,2,8,3,20,0,0
5528,12924,01-04-18,8.000000,Awendo,Nairobi,Bus,49,15,2018-01-04 08:00:00,3,...,1,2,1,2,1,2,1,2,0,0


In [ ]:
distance = {'Migori': 370 , 'Keroka': 280, 'Homa Bay':360, 'Kisii':305.1, 'Keumbu':295, 'Rongo':332,
'Kijauri':271, 'Oyugis':330.6, 'Awendo':351, 'Sirare':392, 'Nyachenge':326, 'Kehancha': 387.7,
'Kendu Bay':347, 'Sori':399, 'Rodi':348, 'Mbita':401, 'Ndhiwa': 371}
data["travel_from_distance"]=data.travel_from.map(distance)

In [ ]:
time = {'Migori': 7*60+8 , 'Keroka': 5*60, 'Homa Bay':7*60, 'Kisii':5*60+34, 'Keumbu':5*60+20, 'Rongo':6*60+21,
'Kijauri':60*4+50,'Oyugis':5*60+50, 'Awendo':6*60+38, 'Sirare':7*60+30, 'Nyachenge':6*60+10, 'Kehancha':7*60+10,
'Kendu Bay':6*60+10, 'Sori':7*60+30, 'Rodi':6*60+40, 'Mbita':7*60+23, 'Ndhiwa': 7*60}
data["travel_from_time"]=data.travel_from.map(time)

In [ ]:
data["Speed"]=data.travel_from_time/data.travel_from_distance

Encoding Categorical features

In [ ]:
time = {'Migori': 7*60+8 , 'Keroka': 5*60, 'Homa Bay':7*60, 'Kisii':5*60+34, 'Keumbu':5*60+20, 'Rongo':6*60+21,
'Kijauri':60*4+50,'Oyugis':5*60+50, 'Awendo':6*60+38, 'Sirare':7*60+30, 'Nyachenge':6*60+10, 'Kehancha':7*60+10,
'Kendu Bay':6*60+10, 'Sori':7*60+30, 'Rodi':6*60+40, 'Mbita':7*60+23, 'Ndhiwa': 7*60}
for key in time.keys():
    time[key]=timedelta( minutes=time[key])

#find the date of traveling from origin
data["Date_of_traveling_from_origin"]=data.travel_from.map(time)

# find the date of arrival at distination
data["arrival_date"]=data.date+ data.Date_of_traveling_from_origin
data["hod_arrived_date"]=data["arrival_date"].dt.hour
data["minute_arrived_date"]=data["arrival_date"].dt.minute
del data["Date_of_traveling_from_origin"],data["arrival_date"]
data["is_rush_hour"]=0
data.loc[data.hod_arrived_date.between(7,17),"is_rush_hour"]=1

In [ ]:
from sklearn import preprocessing #Import LabelEncoder
data = pd.get_dummies(data, columns=['travel_from','day_of_month','month'])
label_enc = {'Bus':1,'shuttle':0}
data.replace(label_enc,inplace=True)

In [ ]:
not_used_cols=["ride_id","travel_time" ,"travel_date","date",'year_woy','max_capacity','number_of_ticket',
               'period'
]

In [ ]:
features = data.drop(not_used_cols,axis=1).columns
features

Index(['travel_to', 'car_type', 'day_of_week', 'day_of_year', 'hour', 'minute',
       'is_weekend', 'year', 'quarter', 'hourly_travelers', 'daily_travelers',
       'diff_btw_0_1_next_bus', 'diff_btw_0_1_previous_bus',
       'diff_btw_0_2_next_bus', 'diff_btw_0_2_previous_bus',
       'diff_btw_0_3_next_bus', 'diff_btw_0_3_previous_bus',
       'diff_btw_next_previous_bus', 'count_trip_per_1min_travel_from',
       'count_trip_per_1min', 'count_trip_per_3min_travel_from',
       'count_trip_per_3min', 'count_trip_per_5min_travel_from',
       'count_trip_per_5min', 'count_trip_per_7min_travel_from',
       'count_trip_per_7min', 'count_trip_per_10min_travel_from',
       'count_trip_per_10min', 'count_trip_per_15min_travel_from',
       'count_trip_per_15min', 'is_holidays', 'is_after_toomorw_holidays',
       'travel_from_distance', 'travel_from_time', 'Speed', 'hod_arrived_date',
       'minute_arrived_date', 'is_rush_hour', 'travel_from_Awendo',
       'travel_from_Homa Bay', 'tra

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [ ]:
data[features]


,travel_to,car_type,day_of_week,day_of_year,hour,minute,is_weekend,year,quarter,hourly_travelers,...,travel_from_Rodi,travel_from_Rongo,travel_from_Sirare,travel_from_Sori,day_of_month_1,day_of_month_2,day_of_month_3,month_1,month_2,month_3
502,Nairobi,1,4,132,7,8,0,2017,2,8.494743,...,0,0,0,0,1,0,0,0,0,1
5530,Nairobi,1,3,4,7,8,0,2018,1,8.494743,...,0,0,0,0,1,0,0,1,0,0
5533,Nairobi,1,3,4,7,12,0,2018,1,8.494743,...,0,0,0,0,1,0,0,1,0,0
5539,Nairobi,1,3,4,7,13,0,2018,1,8.494743,...,0,0,0,0,1,0,0,1,0,0
5528,Nairobi,1,3,4,8,0,0,2018,1,8.494743,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5833,Nairobi,1,3,277,7,5,0,2018,4,8.494743,...,0,0,0,1,1,0,0,0,0,1
5875,Nairobi,1,6,308,7,5,1,2018,4,8.494743,...,0,0,0,1,1,0,0,0,1,0
5834,Nairobi,1,6,308,23,10,1,2018,4,5.659482,...,0,0,0,1,1,0,0,0,1,0
5921,Nairobi,1,1,338,7,5,0,2018,4,8.494743,...,0,0,0,1,1,0,0,1,0,0


In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler =StandardScaler()







# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# scaler.fit(X_test)
# X_test = scaler.transform(X_test)

used_cols = ['day_of_year']
prev_adjR2 = -0.01177282976168792
for col in features:
  used_cols.append(col)
  dataset_pr = data[used_cols]
  X= dataset_pr.copy()
  y = data['number_of_ticket']
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)
  reg = (random_state=0)
  reg.fit(X_train, y_train)
  # Predicting
  y_pred = reg.predict(X_test)
  #Find R-squared value
  r2 = r2_score(y_test, y_pred)
  # Find Adjusted R-squared value
  adj_r2=1-(1-r2_score(y_test, y_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
  if prev_adjR2<adj_r2:
    prev_adjR2 = adj_r2
    train_score = reg.score(X_train, y_train)
    test_score = reg.score(X_test,y_test)
  else:
    used_cols.pop()

print(f'Adjust_r2: {prev_adjR2}')
print(f'Train score: {train_score}')
print(f'Test score: {test_score}')
print('Important features are: \n')
used_cols

In [ ]:
import xgboost as xgb
used_cols = [ 'diff_btw_0_1_next_bus','diff_btw_0_2_next_bus','diff_btw_0_2_previous_bus','count_trip_per_5min_travel_from',
             'hod_arrived_date','travel_from_distance','minute_arrived_date','Speed','travel_from_Sirare'
             ]
# dataset_pr = data[]
X = data[used_cols].copy()
y = data['number_of_ticket'].copy()

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)
dreg= xgb.XGBRegressor(
                        booster= 'gbtree',
                        colsample_bylevel= 1,
                        colsample_bynode= 1,
                        colsample_bytree= 0.7,
                        eta= 0.004,
                        gamma= 0,
                        importance_type= 'gain',
                        learning_rate= 0.1,
                        max_delta_step= 0,
                        max_depth= 9,
                        min_child_weight= 10,
                        n_estimators= 100,
                        n_jobs= 1,
                        objective= 'reg:linear',
                        random_state= 0,
                        reg_alpha= 0,
                        reg_lambda= 1,
                        scale_pos_weight= 1,
                        subsample= 1,
                        verbosity= 1)
dreg.fit(X_train, y_train)
y_pred = dreg.predict(X_test)
#Find R-squared value
r2 = r2_score(y_test, y_pred)
# Find Adjusted R-squared value
adj_r2=1-(1-r2_score(y_test, y_pred))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
train_score = dreg.score(X_train, y_train)
test_score = dreg.score(X_test,y_test)
print(f'Train score: {train_score}')
print(f'Test score: {test_score}')
print('Mean Squared Error (MSE): ',mean_squared_error(y_test, y_pred))
print('Mean Absolute Error (MAE): ',mean_absolute_error(y_test,y_pred))
print('Root Mean Squared Error (MSE): ',np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2: ',r2)
print('Adjusted R2: ',adj_r2)

[08:07:40] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
Train score: 0.7690415054900086
Test score: 0.6646038571553754
Mean Squared Error (MSE):  25.171156455346043
Mean Absolute Error (MAE):  3.194346902588478
Root Mean Squared Error (MSE):  5.0170864508543245
R2:  0.6646038571553754
Adjusted R2:  0.6631328214411446
